In [ ]:
import pandas as pd
import spacy
import umap
import numpy as np 
from io import BytesIO
from PIL import Image
import base64
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10, Category10
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

from ml_editor.data_processing import format_raw_df, get_split_by_author, get_normalized_series, add_text_features_to_df
from ml_editor.data_visualization import plot_embeddings

df = pd.read_csv(Path('../data/writers.csv'))
df = format_raw_df(df.copy())

train_author, test_author = get_split_by_author(df[df["is_question"]])

questions = train_author[train_author["is_question"]]
raw_text = questions["body_text"]
sent_labels = questions["AcceptedAnswerId"].notna()

nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner", 
                                            "textcat", "lemmatizer"])

spacy_emb = train_author[train_author["is_question"]]["body_text"].apply(lambda x: nlp(x).vector)
embeddings = np.vstack(spacy_emb)

umap_embedder = umap.UMAP()
umap_emb = umap_embedder.fit_transform(embeddings)